In [1]:
import nltk
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
from nltk.corpus import stopwords

In [2]:
def getDataSetURL(file_name):
    url = []
    i = 0
    with open(file_name) as filecsv:
        reader = csv.reader(filecsv, delimiter=',')
        for row in reader:
            if i != 0:
                url.append(str(row[0]))
            i += 1

    return url

In [3]:
page_link = getDataSetURL('DataSetArtikel.csv')
article = []
for data in page_link:
    page_response = requests.get(data, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    findP = page_content.find_all("p")

    textContent = []
    for i in range(0, len(findP)):
        paragraphs = findP[i].text
        textContent.append(paragraphs)

    a = []
    i = 0
    findAt = 0
    for data in textContent:
        if 'Klikdokter.com' in data:
            findAt = i
        i += 1

    for i in range(findAt, len(textContent)):
        a.append(textContent[i])

    article.append(a)
np.savetxt('article.txt', article, fmt="%s")

In [4]:
data = []
file = open('article.txt')
spamreader = file.readlines()
for row in spamreader:
    data.append(row.lower()) #mengecilkan huruf kapital
print(data)


['[\'klikdokter.com, jakarta menjelang tahun baru imlek, sejumlah pusat perbelanjaan, restoran, atau rumah-rumah masyarakat tionghoa dipenuhi dengan dekorasi serba merah. warna merah melambangkan semangat, keberanian, keberuntungan, dan kesejahteraan. lalu, adakah pengaruh melihat warna merah terhadap tubuh dan otak manusia?\', \'sejarah warna merah yang identik dengan imlek ini berawal dari sebuah legenda cina tentang nian, seekor binatang buas yang meneror penduduk desa pada tahun baru dan suka memangsa hasil perkebunan, ternak, bahkan anak-anak. penduduk desa percaya bahwa nian takut akan warna merah. sejak itulah warna merah tak bisa dipisahkan dengan imlek.\', \'baca juga\', \'di balik itu, ternyata warna merah bisa memberikan pengaruh tertentu terhadap tubuh dan otak manusia. apa saja efeknya?\', \'sebuah penelitian yang dimuat di jurnal appetite pada tahun 2012 menyebutkan, partisipan yang diberikan makanan dengan piring merah cenderung makan lebih sedikit dibandingkan partisipa

In [28]:
import re
from nltk.tokenize import RegexpTokenizer

#tokenisasi
training_set = data
training_set_temp = []
HEADER = 'klikdokter.com'

data_tokenisasi, hasil_tokenisasi = [], []
# tanda_baca = ['"','?','rs/',',','--','.','(',')',"'",'``',"''",'-','[',']','©','%','\\xa0','“','rn/','\\xa0sinus\\xa0dan','\\xa0tim','\\xa0social','nb/','rvs'] #untuk menghilangkan tanda baca

# Hapus header untuk tiap artikel
for i in training_set:
    temp = i.replace(HEADER, ' ')
    training_set_temp.append(temp)
    
training_set = training_set_temp

tokenizer = RegexpTokenizer(r'\w+')
for i in training_set:
    data_tokenisasi.append(tokenizer.tokenize(i))

In [21]:
#membuat unigram menggunakan dictionary
#membuat frekuensi 

kamus = {}
jum_kata_kamus = 0

for j in data_tokenisasi: #menghitung kata-kata yang ada berdasarkan corpus dengan menghitung jumlah keseluruhan kata
    for k in j:
        if (k in kamus): #menghitung jumlah kata yang sudah di dalam kamus
            kamus[k] +=1
        else:
            kamus[k] = 1 #menghitung jumlah kata yang belum di dalam kamus
            jum_kata_kamus += 1 
        # mengetahui jumlah seluruh kata
        
print(kamus)
print(jum_kata_kamus)

{'jakarta': 113, 'menjelang': 5, 'tahun': 106, 'baru': 50, 'imlek': 5, 'sejumlah': 36, 'pusat': 11, 'perbelanjaan': 1, 'restoran': 4, 'atau': 380, 'rumah': 59, 'masyarakat': 37, 'tionghoa': 1, 'dipenuhi': 3, 'dengan': 626, 'dekorasi': 3, 'serba': 6, 'merah': 50, 'warna': 44, 'melambangkan': 1, 'semangat': 5, 'keberanian': 1, 'keberuntungan': 1, 'dan': 1077, 'kesejahteraan': 4, 'lalu': 36, 'adakah': 3, 'pengaruh': 5, 'melihat': 26, 'terhadap': 77, 'tubuh': 332, 'otak': 50, 'manusia': 30, 'sejarah': 2, 'yang': 1770, 'identik': 3, 'ini': 539, 'berawal': 2, 'dari': 454, 'sebuah': 74, 'legenda': 1, 'cina': 5, 'tentang': 20, 'nian': 2, 'seekor': 1, 'binatang': 5, 'buas': 1, 'meneror': 1, 'penduduk': 7, 'desa': 2, 'pada': 364, 'suka': 6, 'memangsa': 1, 'hasil': 24, 'perkebunan': 2, 'ternak': 1, 'bahkan': 65, 'anak': 107, 'percaya': 16, 'bahwa': 215, 'takut': 6, 'akan': 292, 'sejak': 28, 'itulah': 7, 'tak': 196, 'bisa': 384, 'dipisahkan': 1, 'baca': 99, 'juga': 432, 'di': 707, 'balik': 4, 'itu

In [7]:
#baru mencari probabilitas unigram 
#memakai dictionary agar membaca lebih gampang berdasarkan key dan value
probabilitas_unigram = {} 

for x in kamus.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    probabilitas_unigram[x] = kamus[x]/jum_kata_kamus
print (probabilitas_unigram)

{'jakarta': 0.019172039362063114, 'menjelang': 0.0008483203257550051, 'tahun': 0.017984390906006106, 'baru': 0.00848320325755005, 'imlek': 0.0008483203257550051, 'sejumlah': 0.006107906345436036, 'pusat': 0.0018663047166610112, 'perbelanjaan': 0.00016966406515100103, 'restoran': 0.0006786562606040041, 'atau': 0.06447234475738038, 'rumah': 0.01001017984390906, 'masyarakat': 0.006277570410587038, 'tionghoa': 0.00016966406515100103, 'dipenuhi': 0.0005089921954530031, 'dengan': 0.10620970478452664, 'dekorasi': 0.0005089921954530031, 'serba': 0.0010179843909060061, 'merah': 0.00848320325755005, 'warna': 0.007465218866644045, 'melambangkan': 0.00016966406515100103, 'semangat': 0.0008483203257550051, 'keberanian': 0.00016966406515100103, 'keberuntungan': 0.00016966406515100103, 'dan': 0.1827281981676281, 'kesejahteraan': 0.0006786562606040041, 'lalu': 0.006107906345436036, 'adakah': 0.0005089921954530031, 'pengaruh': 0.0008483203257550051, 'melihat': 0.004411265693926026, 'terhadap': 0.013064

In [8]:
#menghitung bigram 
kamus_bigram = {}
jum_kata_kamus_bigram = 0

for j in (data_tokenisasi): #menghitung kata-kata yang ada berdasarkan corpus dengan menghitung jumlah keseluruhan kata
    for k in range(len(j)-1):
        if ((j[k],j[k+1]) in kamus_bigram): #menghitung jumlah kata yang sudah di dalam kamus
            kamus_bigram[(j[k], j[k+1])] +=1
        else:
            kamus_bigram[(j[k], j[k+1])] = 1 #menghitung jumlah kata yang belum di dalam kamus
            # mengetahui jumlah seluruh kata
            jum_kata_kamus_bigram += 1 
        
print((kamus_bigram))
print(jum_kata_kamus_bigram)

{('jakarta', 'menjelang'): 2, ('menjelang', 'tahun'): 1, ('tahun', 'baru'): 10, ('baru', 'imlek'): 1, ('imlek', 'sejumlah'): 1, ('sejumlah', 'pusat'): 1, ('pusat', 'perbelanjaan'): 1, ('perbelanjaan', 'restoran'): 1, ('restoran', 'atau'): 1, ('atau', 'rumah'): 3, ('rumah', 'rumah'): 1, ('rumah', 'masyarakat'): 1, ('masyarakat', 'tionghoa'): 1, ('tionghoa', 'dipenuhi'): 1, ('dipenuhi', 'dengan'): 1, ('dengan', 'dekorasi'): 1, ('dekorasi', 'serba'): 1, ('serba', 'merah'): 2, ('merah', 'warna'): 1, ('warna', 'merah'): 20, ('merah', 'melambangkan'): 1, ('melambangkan', 'semangat'): 1, ('semangat', 'keberanian'): 1, ('keberanian', 'keberuntungan'): 1, ('keberuntungan', 'dan'): 1, ('dan', 'kesejahteraan'): 3, ('kesejahteraan', 'lalu'): 1, ('lalu', 'adakah'): 1, ('adakah', 'pengaruh'): 1, ('pengaruh', 'melihat'): 1, ('melihat', 'warna'): 3, ('merah', 'terhadap'): 1, ('terhadap', 'tubuh'): 3, ('tubuh', 'dan'): 18, ('dan', 'otak'): 4, ('otak', 'manusia'): 3, ('manusia', 'sejarah'): 1, ('sejarah

In [10]:
#menghitung probabilitas bigram 

#jumlah dari key (0,0+1)/jumlah(key(0))

probabilitas_bigram = {} 

for x in kamus_bigram.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    kata0 = x[0]
    kata1 = x[1]
    probabilitas_bigram[(kata0,kata1)] = kamus_bigram[(kata0,kata1)]/kamus[kata0]
print (probabilitas_bigram)

{('jakarta', 'menjelang'): 0.017699115044247787, ('menjelang', 'tahun'): 0.2, ('tahun', 'baru'): 0.09433962264150944, ('baru', 'imlek'): 0.02, ('imlek', 'sejumlah'): 0.2, ('sejumlah', 'pusat'): 0.027777777777777776, ('pusat', 'perbelanjaan'): 0.09090909090909091, ('perbelanjaan', 'restoran'): 1.0, ('restoran', 'atau'): 0.25, ('atau', 'rumah'): 0.007894736842105263, ('rumah', 'rumah'): 0.01694915254237288, ('rumah', 'masyarakat'): 0.01694915254237288, ('masyarakat', 'tionghoa'): 0.02702702702702703, ('tionghoa', 'dipenuhi'): 1.0, ('dipenuhi', 'dengan'): 0.3333333333333333, ('dengan', 'dekorasi'): 0.001597444089456869, ('dekorasi', 'serba'): 0.3333333333333333, ('serba', 'merah'): 0.3333333333333333, ('merah', 'warna'): 0.02, ('warna', 'merah'): 0.45454545454545453, ('merah', 'melambangkan'): 0.02, ('melambangkan', 'semangat'): 1.0, ('semangat', 'keberanian'): 0.2, ('keberanian', 'keberuntungan'): 1.0, ('keberuntungan', 'dan'): 1.0, ('dan', 'kesejahteraan'): 0.002785515320334262, ('kesej

In [29]:
import math

def get_perplexity(prob_bigram):
    pp = 1
    for key in prob_bigram.keys():
        # Menghindari underflow menggunakan log
        pp += math.log(prob_bigram[key])
    pp = pp ** (1/len(prob_bigram))
    return pp

In [37]:
# Menguji perplexity terhadap test set

test_set = data[11:15]
tokenisasi_test = []

tokenizer = RegexpTokenizer(r'\w+')
for i in test_set:
    tokenisasi_test.append(tokenizer.tokenize(i))
    
# print(tokenisasi_test)

In [22]:
# memprediksi kata yang muncul setelah kalimat yang diinputkan
kamus_temp = kamus
for artikel in tokenisasi_test:
    for kata in artikel:
        if kata not in kamus:
            kamus_temp[kata]= 1

prob_unigram_temp = {}
print(len(kamus_temp))
for key in kamus_temp.keys():
    prob_unigram_temp[key] = kamus_temp[key]/len(kamus_temp)

5894


In [23]:
# Laplace-smoothed bigram counts

kamus_bigram_temp = {}
keys = list(kamus_temp.keys())

for i in range(len(keys)):
    for j in range(len(keys)):
        if((keys[i], keys[j]) in kamus_bigram):
            kamus_bigram_temp[(keys[i], keys[j])] = kamus_bigram[(keys[i], keys[j])] + 1
        else:
            kamus_bigram_temp[(keys[i], keys[j])] = 1

In [24]:
print(len(kamus_bigram_temp))

34739236


In [35]:
# Probabilitas bigram setelah di smoothing

prob_bigram_temp = {} 

for x in kamus_bigram_temp.keys():
    #menghitung jumlah kata yang dihitung dengan jumlah kata keseluruhan 
    kata0 = x[0]
    kata1 = x[1]
    prob_bigram_temp[(kata0,kata1)] = kamus_bigram_temp[(kata0,kata1)]/(kamus_temp[kata0] + len(kamus_temp))

In [36]:
# # Menampilkan bigram yang paling sering muncul
# import operator
# sorted(prob_bigram_temp.items(), key=operator.itemgetter(1), reverse=True)

In [27]:
# Hitung perplexity
print(get_perplexity(prob_bigram_temp))

(1.0000005620354977+9.043360709707009e-08j)


In [34]:
#memprediksi kata yang muncul setelah 2 kata yang diinputkan
array_kata2 = []
nilai_prob_kata2 = []

minta_inputan = input().lower().split()
print(minta_inputan)

for y in probabilitas_bigram.keys(): #melakukan looping untuk melihat keys yang ada di porbabilitas bigram 
    #melakukan pengecekan 
    if (minta_inputan[1] == y[0]): #ketika sama akan dimasukkan ke array baru
        array_kata2.append(y[1])
        nilai_prob_kata2.append(probabilitas_bigram[(minta_inputan[1], y[1])])
#     else:
        
maksimum = nilai_prob_kata2[0]
for looping2 in range(len(nilai_prob_kata2)): #bisa menyamakan indeksnya
    if (maksimum < nilai_prob_kata2[looping2]):
        maksimum = nilai_prob_kata2[looping2]
        kata_tetap = array_kata2[looping2]
    
print(maksimum) #mengeluarkan probabilitas terbaik atau maksimum dari kata2 yang telah dicari didalam bigram
print(kata_tetap) #kata teteap merupakan mengeluarkan kata yang dikeluarkan setelah kata yang diinputkan
print(array_kata2) #mengeluarkan kata-kata selanjutnya yang berhubungan dengan per-kata yang diinputkan 
        

Sama juga
['sama', 'juga']
0.12731481481481483
dapat
['di', 'halnya', 'menyebutkan', 'apa', 'menjelaskan', 'kerap', 'dapat', 'dilansir', 'pemeriksaan', 'perlu', 'diperlukan', 'kelima', 'bisa', 'tidak', 'memiliki', 'cukup', 'kaya', 'mengandung', 'untuk', 'wanita', 'khawatir', 'pertama', 'rentan', 'ada', 'ya', 'akan', 'disetujui', 'seorang', 'mampu', 'diwajibkan', 'harus', 'penasaran', 'diet', 'sering', 'tak', 'digunakan', 'terkandung', 'percaya', 'selain', 'adanya', 'mengukur', 'menemukan', 'membuat', 'kadang', 'berikut', 'sangat', 'beberapa', 'berkhasiat', 'sumatera', 'bereaksi', 'lho', 'mirisnya', 'mengonsumsi', 'menurut', 'yang', 'menghasilkan', 'seperti', 'merupakan', 'terbukti', 'menarik', 'bersifat', 'tapi', 'pemberian', 'masih', 'mewarnai', 'turut', 'menurun', 'mengindikasikan', 'coba', 'karena', 'dijuluki', 'membawa', 'dengan', 'memberikan', 'pengetahuan', 'semakin', 'jika', 'membantu', 'kapan', 'apapun', 'biasanya', 'berpengaruh', 'dibiarkan', 'bagi', 'banyak', 'berlaku', 'lebi